In [1]:
# backpropagation_4_12 (Solution to Exercise 4.12)
# Author: Matthew Dixon
# Version: 1.0 (24.7.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6 and Tensorflow 1.3.0
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Bilokon P., Dixon M.F. and I. Halperin, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 
# This notebook is courtesy of Justin Li, Imperial College (jcl514@ic.ac.uk)

In [1]:
import numpy as np
import scipy as sp
import pandas as pd 
from numpy.linalg import norm
import copy
import os

In [3]:
def relu(x):
    return x*(np.sign(x)+1.)/2.

In [4]:
def heaviside(x):
    return (np.sign(x)+1.)/2

In [5]:
def sigmoid(x):
    return 1./(1.+np.exp(-x))

In [6]:
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [7]:
def mynorm(Z):
    return np.sqrt(np.mean(Z**2))

Let us consider a feed-forward architecture with an input layer, $L-1$ hidden layers and one output layer, with $K$ units in the output layer. As a result, we have $L$ sets of weights and biases $(W^{(\ell)}, \mathbf{b}^{(\ell)})$ for $\ell=1,\dots, L$, corresponding to the layer inputs $Z^{(\ell-1)}$ and outputs $Z^{(\ell)}$ for $\ell=1,\dots, L$. Recall that each layer is an activation of a semi-affine transformation, $I^{(\ell)}(Z^{(\ell-1)}):=W^{(L)}Z^{(\ell-1)}+ b^{(L)}$. The corresponding activation functions are denoted as $\sigma^{(\ell)}$. The activation function for the output layer is a softmax function, $\sigma_s(x)$. 

Here we use the cross-entropy as the loss function, which is defined as 
$$ \mathcal{L}:= -\sum_{k=1}^{K}Y_{k}\log \hat{Y}_{k}.$$

The relationship between the layers, for $\ell\in\{1,\dots, L\}$ are:

\begin{align*}
\hat{Y} (X) & = Z^{(L)}=\sigma_s(I^{(L)}) \in [0,1]^{K},\\
Z^{(\ell)} & = \sigma^{(\ell)} \left ( I^{(\ell)} \right ), ~\ell=1,\dots,L-1,\\
Z^{(0)} & = X.\\
\end{align*}


The update rules for the weights and biases are
\begin{align*}
    \Delta W^{(\ell)} &=  - \gamma \nabla_{W^{(\ell)}}\mathcal{L},\\
    \Delta \mathbf{b}^{(\ell)} &=  - \gamma \nabla_{\mathbf{b}^{(\ell)}}\mathcal{L}.
\end{align*}
We now begin the Back-Propagation.

For the gradient of $\mathcal{L}$ w.r.t. $W^{(L)}$ we have
\begin{align*}
    \frac{\partial \mathcal{L}}{\partial w_{ij}^{(L)}} &= \sum_{k=1}^{K}\frac{\partial \mathcal{L}}{\partial Z_{k}^{(L)}} \frac{\partial Z_{k}^{(L)}}{\partial w_{ij}^{(L)}}\\
    %%%%%%%%%%%
    &= \sum_{k=1}^{K}\frac{\partial \mathcal{L}}{\partial Z_{k}^{(L)}} \sum_{m=1}^{K}\frac{\partial Z_{k}^{(L)}}{\partial I_{m}^{(L)}} \frac{\partial I_{m}^{(L)}}{\partial w_{ij}^{(L)}}
\end{align*}
But 
\begin{align*}
    \frac{\partial \mathcal{L}}{\partial Z_{k}^{(L)}} &= -\frac{Y_{k}}{Z_{k}^{(L)}}\\
    %%%%%%%%%%%%%%
    \frac{\partial Z_{k}^{(L)}}{\partial I_{m}^{(L)}} &= \frac{\partial}{\partial I_{m}^{(L)}}[\sigma(I^{(L)})]_{k}\\
    %%%%%%%%%%%%%%
    &= \frac{\partial}{\partial I_{m}^{(L)}} \frac{\exp[I_{k}^{(L)}]}{\sum_{n=1}^{K}\exp[I_{n}^{(L)}]}\\
    %%%%%%%%%%%%%%
    &= \begin{cases} 
    -\frac{\exp[I_{k}^{(L)}]}{\sum_{n=1}^{K}\exp[I_{n}^{(L)}]} \frac{\exp[I_{m}^{(L)}]}{\sum_{n=1}^{K}\exp[I_{n}^{(L)}]} & \text{if } k \neq m \\
    \frac{\exp[I_{k}^{(L)}]}{\sum_{n=1}^{K}\exp[I_{n}^{(L)}]} - \frac{\exp[I_{k}^{(L)}]}
    {\sum_{n=1}^{K}\exp[I_{n}^{(L)}]} \frac{\exp[I_{m}^{(L)}]}{\sum_{n=1}^{K}\exp[I_{n}^{(L)}]} 
    & \text{otherwise}
    \end{cases}\\
    %%%%%%%%%%%%%%
    &= \begin{cases} 
    -\sigma_{k}\sigma_{m}& \text{if } k \neq m \\
     \sigma_k(1 - \sigma_m) & \text{otherwise}
    \end{cases}\\
    %%%%%%%%%%%%%%
    &= \sigma_k(\delta_{km} - \sigma_m) \quad \text{where} \, \delta_{km} \, \text{is the Kronecker's Delta}\\
    %%%%%%%%%%%%%%
    \frac{\partial I_{m}^{(L)}}{\partial w_{ij}^{(L)}} &= \delta_{mi}Z_{j}^{(L-1)}\\
    %%%%%%%%%%%%%%
    \implies \frac{\partial \mathcal{L}}{\partial w_{ij}^{(L)}} &= -\sum_{k=1}^{K}\frac{Y_{k}}{Z_{k}^{(L)}} 
    \sum_{m=1}^{K} Z_{m}^{(L)}(\delta_{km} - Z_{m}^{(L)}) \delta_{mi}Z_{j}^{(L-1)}\\
    %%%%%%%%%%%%%%
    &= -Z_{j}^{(L-1)} \sum_{k=1}^{K}Y_{k}  (\delta_{ki} - Z_{i}^{(L)}) \\
    %%%%%%%%%%%%%%
    &= Z_{j}^{(L-1)} (Z_{i}^{(L)}-Y_{i})
\end{align*}
Where we have used the fact that $\sum_{k=1}^{K}Y_{k}=1$ in the last equality.

Similarly for $\mathbf{b}^{(L)}$, we have
\begin{align*}
    \frac{\partial \mathcal{L}}{\partial b_{i}^{(L)}} &= \sum_{k=1}^{K}\frac{\partial \mathcal{L}}{\partial Z_{k}^{(L)}} \sum_{m=1}^{K}\frac{\partial Z_{k}^{(L)}}{\partial I_{m}^{(L)}} \frac{\partial I_{m}^{(L)}}{\partial b_{i}^{(L)}}\\
    %%%%%%%%%%%%%%%%%%
    &= Z_{i}^{(L)}-Y_{i}
\end{align*}
It follows that
\begin{align*}
    \nabla_{\mathbf{b}^{(L)}}\mathcal{L} &= Z^{(L)}-Y\\
    \nabla_{W^{(L)}}\mathcal{L} &= \nabla_{\mathbf{b}^{(L)}}\mathcal{L} \otimes {Z^{(L-1)}}
\end{align*}
Where $\otimes$ denotes the outer product.

For the gradient of $\mathcal{L}$ w.r.t. $W^{(L-1)}$ we have
\begin{align*}
    \frac{\partial \mathcal{L}}{\partial w_{ij}^{(L-1)}} &= \sum_{k=1}^{K}\frac{\partial L}{\partial Z_{k}^{(L)}} \frac{\partial Z_{k}^{(L)}}{\partial w_{ij}^{(L-1)}}\\
    %%%%%%%%%%%
    &= \sum_{k=1}^{K}\frac{\partial \mathcal{L}}{\partial Z_{k}^{(L)}} \sum_{m=1}^{K}\frac{\partial Z_{k}^{(L)}}{\partial I_{m}^{(L)}} \sum_{n=1}^{n^{(L-1)}} \frac{\partial I_{m}^{(L)}}{\partial Z_{n}^{(L-1)}} \sum_{p=1}^{n^{(L-1)}} \frac{\partial Z_{n}^{(L-1)}}{\partial I_{p}^{(L-1)}} \frac{\partial I_{p}^{(L-1)}}{\partial w_{ij}^{(L-1)}}
\end{align*}
If we assume that $\sigma^{(\ell)}(x)=\text{sigmoid}(x), ~\ell \in \{1,\dots, L-1\}$ then 
\begin{align*}
    \frac{\partial I_{m}^{(L)}}{\partial Z_{n}^{(L-1)}} &= w_{mn}^{(L)}\\
    %%%%%%%%%%%%%%%%%
    \frac{\partial Z_{n}^{(L-1)}}{\partial I_{p}^{(L-1)}} &= \frac{\partial}{\partial I_{p}^{(L-1)}}\bigg(\frac{1}{1+\exp(-I_{n}^{(L-1)})}\bigg)\\
    %%%%%%%%%%%%%%%%%
    &= \frac{1}{1+\exp(-I_{n}^{(L-1)})} \frac{\exp(-I_{n}^{(L-1)})}{1+\exp(-I_{n}^{(L-1)})} \, \delta_{np} \\
    %%%%%%%%%%%%%%%%%
    &= Z_{n}^{(L-1)} (1-Z_{n}^{(L-1)}) \, \delta_{np} = \sigma^{(L-1)}_n(1-\sigma^{(L-1)}_n)\delta_{np} \\
    %%%%%%%%%%%%%%%%%
    \frac{\partial I_{p}^{(L-1)}}{\partial w_{ij}^{(L-1)}} &= \delta_{pi} Z_{j}^{(L-2)} \\
    %%%%%%%%%%%%%%%%%
    \implies \frac{\partial L}{\partial w_{ij}^{(L)}} &= -\sum_{k=1}^{K}\frac{Y_{k}}{Z_{k}^{(L)}} \sum_{m=1}^{K}Z_{k}^{(L)}(\delta_{km} - Z_{m}^{(L)}) \sum_{n=1}^{n^{(L-1)}} w_{mn}^{(L)} \sum_{p=1}^{n^{(L-1)}} Z_{n}^{(L-1)} (1-Z_{n}^{(L-1)}) \, \delta_{np} \delta_{pi} Z_{j}^{(L-2)} \\
    %%%%%%%%%%%%%%%%%
    &= -\sum_{k=1}^{K}Y_{k} \sum_{m=1}^{K}(\delta_{km} - Z_{m}^{(L)}) \sum_{n=1}^{n^{(L-1)}} w_{mn}^{(L)} Z_{n}^{(L-1)} (1-Z_{n}^{(L-1)}) \, \delta_{ni} Z_{j}^{(L-2)} \\
    %%%%%%%%%%%%%%%%%
    &= -\sum_{k=1}^{K}Y_{k} \sum_{m=1}^{K}(\delta_{km} - Z_{m}^{(L)}) w_{mi}^{(L)} Z_{i}^{(L-2)} (1-Z_{i}^{(L-1)}) Z_{j}^{(L-2)} \\
    %%%%%%%%%%%%%%%%%
    &= -Z_{j}^{(L-2)}Z_{i}^{(L-1)}(1-Z_{i}^{(L-1)}) \sum_{m=1}^{K} w_{mi}^{(L)} \sum_{k=1}^{K}(\delta_{km}Y_{k} - Z_{m}^{(L)}Y_{k}) \\
    %%%%%%%%%%%%%%%%%
    &= Z_{j}^{(L-2)}Z_{i}^{(L-1)} (1-Z_{i}^{(L-1)}) \sum_{m=1}^{K} w_{mi}^{(L)} (Z_{m}^{(L)} - Y_{m}) \\
    %%%%%%%%%%%%%%%%%
    &= Z_{j}^{(L-2)}Z_{i}^{(L-1)} (1-Z_{i}^{(L-1)}) (Z^{(L)} - Y)^{T} \mathbf{w}_{,i}^{(L)} \\
\end{align*}
Similarly we have
$$ \frac{\partial \mathcal{L}}{\partial b_{i}^{(L-1)}} = Z_{i}^{(L-1)} (1-Z_{i}^{(L-1)}) (Z^{(L)} - Y)^{T} \mathbf{w}_{,i}^{(L)}. $$
It follows that we can define the following recursion relation for the loss gradient:

\begin{align*}
    \nabla_{b^{(L-1)}}\mathcal{L} &= Z^{(L-1)} \circ (\mathbf{1}-Z^{(L-1)}) \circ 
    ({W^{(L)}}^{T} \nabla_{b^{(L)}}\mathcal{L}) \\
    \nabla_{W^{(L-1)}}\mathcal{L} &= \nabla_{b^{(L-1)}}\mathcal{L} \otimes Z^{(L-2)}\\
    & = Z^{(L-1)} \circ (\mathbf{1}-Z^{(L-1)}) \circ 
    ({W^{(L)}}^{T} \nabla_{W^{(L)}}\mathcal{L})
\end{align*}

Where $\circ$ denotes the Hadamard Product (elementwise multiplication). This recursion relation generalizes for all layers. To see this, let the back-propagation error $\delta^{(\ell)}:=\nabla_{b^{(\ell)}}\mathcal{L}$, and since

\begin{align*}
\left[\frac{\partial \sigma^{(\ell)}}{\partial I^{(\ell)}}\right]_{ij}&=\frac{\partial \sigma_i^{(\ell)}}{\partial I_j^{(\ell)}}\\
&=\sigma_i^{(\ell)}(1-\sigma_i^{(\ell)})\delta_{ij}\\
\end{align*}
Equivalently in matrix-vector form
$$\nabla_{I^{(\ell)}} \sigma^{(\ell)}=\text{diag}(\sigma^{(\ell)} \circ (\mathbf{1}-\sigma^{(\ell)})).$$

We can write, in general, for any choice of activation function for the hidden layer,

$$ \delta^{(\ell)}=(\nabla_{I^{(\ell)}} \sigma^{(\ell)})(W^{(\ell+1)})^T\delta^{(\ell+1)}.$$
and

$$\nabla_{W^{(\ell)}}\mathcal{L} = \delta^{(\ell)} \otimes Z^{(\ell-1)}.$$

Implementation of a neural network, outside of Keras, for testing backpropagation.

In [10]:
def myANN(Y,Xtrain,Xpred,W01,W02,W03,b01,b02,b03):
    # Initialization of Weights and Biases
    W1 = copy.copy(W01)
    W2 = copy.copy(W02)
    W3 = copy.copy(W03)
    b1 = copy.copy(b01)
    b2 = copy.copy(b02)
    b3 = copy.copy(b03)
    # Initialize adhoc variables
    k = 1
    change = 999
    # Begin Feedforward (assume learning rate is one)
    while (change > 0.001 and k<201):
        print("Iteration", k)
        # Hidden Layer 1
        Z1 = relu(W1@Xtrain + b1)
        # Hidden Layer 2
        Z2 = sigmoid(W2@Z1 + b2)
        # Output Layer
        Yhat = softmax(W3@Z2 + b3)
        # Find cross-entropy loss
        loss = -Y@np.log(Yhat)
        print("Current Loss:",loss)
        
        # Find gradient of loss with respect to the weights
        # Output Later
        dLdb3 = Yhat - Y #
        dLdW3 = np.outer(dLdb3,Z2)
        # Hidden Layer 2
        dLdb2 = (W3.T@(dLdb3))*Z2*(1-Z2)
        dLdW2 = np.outer(dLdb2,Z1)
        # Hidden Layer 1
        dLdb1 = (W2.T@(dLdb2))*heaviside(W1@Xtrain + b1)
        dLdW1 = np.outer(dLdb1,Xtrain)
        
        # Update Weights by Back Propagation
        # Output Layer
        b3 -= dLdb3 #(learning rate is one)
        W3 -= dLdW3
        # Hidden Layer 2
        b2 -= dLdb2
        W2 -= dLdW2
        # Hidden Layer 1
        b1 -= dLdb1
        W1 -= dLdW1
        
        change = norm(dLdb1)+norm(dLdb2)+norm(dLdb3)+norm(dLdW1)+norm(dLdW2)+norm(dLdW3)
        k+= 1
        
    Z1pred = W1@Xpred + b1
    Z2pred = W2@relu(Z1pred) + b2
    Z3pred = W3@sigmoid(Z2pred) + b3
    Ypred = softmax(Z3pred)
    print("")
    print("Summary")
    print("Target Y", Y)
    print("Fitted Ytrain", Yhat)
    print("Xpred", Xpred)
    print("Fitted Ypred", Ypred)
    print("Weight Matrix 1", W1)
    print("Bias Vector 1", b1)
    print("Weight Matrix 2", W2)
    print("Bias Vector 2", b2)
    print("Weight Matrix 3", W3)
    print("Bias Vector 3", b3)

In [11]:
W0_1 = np.array([[0.1,0.3,0.7],[0.9,0.4,0.4]])
b_1 = np.array([1.,1.])
W0_2 = np.array([[0.4,0.3],[0.7,0.2]])
b_2 = np.array([1.,1.])
W0_3 = np.array([[0.5,0.6],[0.6,0.7],[0.3,0.2]])
b_3 = np.array([1.,1.,1.])
YY = np.array([1.,0.,0.])
X_train = np.array([0.1,0.7,0.3])
X_pred = X_train

In [12]:
myANN(YY,X_train,X_pred,W0_1,W0_2,W0_3,b_1,b_2,b_3)

Iteration 1
Current Loss: 1.0220603290496275
Iteration 2
Current Loss: 0.12835236020870527
Iteration 3
Current Loss: 0.08090078793434305
Iteration 4
Current Loss: 0.05953583479959617
Iteration 5
Current Loss: 0.04721118634517901
Iteration 6
Current Loss: 0.03915120608280936
Iteration 7
Current Loss: 0.03345605316228914
Iteration 8
Current Loss: 0.029213117563831773
Iteration 9
Current Loss: 0.025927552828981213
Iteration 10
Current Loss: 0.02330712020071898
Iteration 11
Current Loss: 0.021167867391973044
Iteration 12
Current Loss: 0.01938811367291565
Iteration 13
Current Loss: 0.017884103003570954
Iteration 14
Current Loss: 0.016596270340631637
Iteration 15
Current Loss: 0.015481081906068636
Iteration 16
Current Loss: 0.014505974042139025
Iteration 17
Current Loss: 0.013646097998312075
Iteration 18
Current Loss: 0.012882160242001632
Iteration 19
Current Loss: 0.012198950566520442
Iteration 20
Current Loss: 0.011584315088619651
Iteration 21
Current Loss: 0.011028424641181308
Iteration 2

# Compare result with Keras

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

Using TensorFlow backend.


In [14]:
# Create Model
model = Sequential()
model.add(Dense(2, input_dim=3, activation='relu', weights = [W0_1.T,b_1]))
model.add(Dense(2, activation='sigmoid', weights = [W0_2.T,b_2]))
model.add(Dense(3, activation='softmax', weights = [W0_3.T,b_3]))
# Compile Model
sgd = optimizers.SGD(lr=1)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_crossentropy'])
model.get_weights()

[array([[0.1, 0.9],
        [0.3, 0.4],
        [0.7, 0.4]], dtype=float32),
 array([1., 1.], dtype=float32),
 array([[0.4, 0.7],
        [0.3, 0.2]], dtype=float32),
 array([1., 1.], dtype=float32),
 array([[0.5, 0.6, 0.3],
        [0.6, 0.7, 0.2]], dtype=float32),
 array([1., 1., 1.], dtype=float32)]

In [15]:
# Fit the model
model.fit(X_train.reshape((1,3)), YY.reshape((1,3)), epochs=200, batch_size=1)

Epoch 1/200
1/1 [==============================] - 0s 175ms/step - loss: 1.0221 - categorical_crossentropy: 1.0221
Epoch 2/200
1/1 [==============================] - 0s 1ms/step - loss: 0.1284 - categorical_crossentropy: 0.1284
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0809 - categorical_crossentropy: 0.0809
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0595 - categorical_crossentropy: 0.0595
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0472 - categorical_crossentropy: 0.0472
Epoch 6/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0392 - categorical_crossentropy: 0.0392
Epoch 7/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0335 - categorical_crossentropy: 0.0335
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0292 - categorical_crossentropy: 0.0292
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0259 - categorical_cr

Epoch 73/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0031 - categorical_crossentropy: 0.0031
Epoch 74/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0031 - categorical_crossentropy: 0.0031
Epoch 75/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0031 - categorical_crossentropy: 0.0031
Epoch 76/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0030 - categorical_crossentropy: 0.0030
Epoch 77/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0030 - categorical_crossentropy: 0.0030
Epoch 78/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0029 - categorical_crossentropy: 0.0029
Epoch 79/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0029 - categorical_crossentropy: 0.0029
Epoch 80/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0029 - categorical_crossentropy: 0.0029
Epoch 81/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0028 - categor

Epoch 145/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0016 - categorical_crossentropy: 0.0016
Epoch 146/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0016 - categorical_crossentropy: 0.0016
Epoch 147/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0015 - categorical_crossentropy: 0.0015
Epoch 148/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0015 - categorical_crossentropy: 0.0015
Epoch 149/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0015 - categorical_crossentropy: 0.0015
Epoch 150/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0015 - categorical_crossentropy: 0.0015
Epoch 151/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0015 - categorical_crossentropy: 0.0015
Epoch 152/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0015 - categorical_crossentropy: 0.0015
Epoch 153/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0015 

In [16]:
model.predict(X_pred.reshape((1,3)))

array([[9.9887174e-01, 5.7524466e-04, 5.5298163e-04]], dtype=float32)

###  Keras Feedforward Results (Note that the results are transposed in order to match Keras' output style)

In [17]:
model.get_weights()

[array([[0.11551907, 0.9084813 ],
        [0.40863392, 0.4593696 ],
        [0.74655724, 0.42544398]], dtype=float32),
 array([1.155191 , 1.0848131], dtype=float32),
 array([[0.6016261 , 0.87797874],
        [0.50208974, 0.37863797]], dtype=float32),
 array([1.1303287, 1.1153796], dtype=float32),
 array([[ 2.0983126 , -0.3175866 , -0.38072622],
        [ 2.2364147 , -0.23999722, -0.49641728]], dtype=float32),
 array([ 2.766754  , -0.01685131,  0.25009623], dtype=float32)]